### PyTorchの基礎

In [1]:
import torch
import torch.nn.functional as F
from torch import nn

/opt/conda/lib/python3.8/site-packages/tqdm/auto.py:22: TqdmWarning: IProgress not found. Please update jupyter and ipywidgets. See https://ipywidgets.readthedocs.io/en/stable/user_install.html
  from .autonotebook import tqdm as notebook_tqdm


- テンソル

In [3]:
t1 = torch.tensor([1, 2, 3, 4])
t2 = torch.zeros(([32, 3, 128, 128]))

print(f't1 = {t1}, t1.shape = {t1.shape}')
print(f't2.shape = {t2.shape}')

t1 = tensor([1, 2, 3, 4]), t1.shape = torch.Size([4])
t2.shape = torch.Size([32, 3, 128, 128])


- テンソルのGPUへの転送

In [4]:
t1 = torch.tensor([1, 2, 3, 4], device='cuda') # 生成時からGPUメモリ上

t2 = torch.tensor([1, 2, 3, 4])
t2 = t2.to('cuda') # 生成はメインメモリ、後からGPUメモリに転送

- Python演算子を使ったテンソルの演算

In [5]:
# 演算は要素ごとに行われる
t1 = torch.tensor([1, 2, 3, 4])
t2 = torch.tensor([2, 4, 6, 8])

t3 = t1 + t2
t4 = t1 ** 2
print(f't3 = {t3}')
print(f't4 = {t4}')

t3 = tensor([ 3,  6,  9, 12])
t4 = tensor([ 1,  4,  9, 16])


- テンソルを処理する関数

In [7]:
# テンソルの形を変形するview関数
# メモリ上のデータ配置と見かけ上の形で整合性が取れない場合view関数使えない
# view関数が使えない場合はreshape関数を使用する
# reshape関数は整合性取れない場合はデータを複製して形を変える
t1 = torch.tensor([1, 2, 3, 4])
t1 = t1.view(2, 2)

t2 = torch.tensor([1, 2, 3, 4, 5, 6])
t2 = t2.view(2, -1)

print(f't1.shape = {t1.shape}')
print(f't2.shape = {t2.shape}')

t1.shape = torch.Size([2, 2])
t2.shape = torch.Size([2, 3])


- transpose関数とpermute関数による軸の並び替え

In [8]:
# transpose関数：任意の2軸を入れ替える
# permute関数はtranspose関数の拡張版　すべての軸を一度に並び替えることが可能
t1 = torch.zeros((32, 3, 128, 128))
t1 = t1.transpose(0, 2)

t2 = torch.zeros((32, 3, 128, 128))
t2 = t2.permute(2, 0, 3, 1)

print(f't1.shape = {t1.shape}')
print(f't2.shape = {t2.shape}')

t1.shape = torch.Size([128, 3, 32, 128])
t2.shape = torch.Size([128, 32, 128, 3])


- cat関数とstack関数による複数テンソルの連結

In [9]:
# cat関数：テンソルが持つ既存の軸の1つで複数のテンソルを連結
# stack関数：新しく軸を追加し、その軸で複数のテンソルを連結
t1 = torch.tensor([1, 2, 3, 4, 5, 6]).view(2, 3)
t2 = torch.tensor([7, 8, 9]).view(1, 3)
t3 = torch.cat((t1, t2))

t4 = torch.tensor([1, 2, 3])
t5 = torch.tensor([4, 5, 6])
t6 = torch.stack((t4, t5), dim=1) # dim 連結する軸　デフォルト0軸（列方向）

print(f't3 = {t3}')
print(f't6 = {t6}')

t3 = tensor([[1, 2, 3],
        [4, 5, 6],
        [7, 8, 9]])
t6 = tensor([[1, 4],
        [2, 5],
        [3, 6]])


- インデクシングによるテンソル要素の抽出

In [11]:
t1 = torch.tensor([1, 2, 3, 4, 5, 6, 7, 8, 9]).view(3, 3)
t2 = t1[[0, 1], :]
t3 = t1[:, [0, 2]]
t4 = t1[[0, 2, 1], [1, 2, 1]] # 0行目1列目、2行目2列目、1行目1列目
t5 = t1[[True, True, False]]
t6 = t1[t1 % 2 == 0]

print(f't2 = {t2}')
print(f't3 = {t3}')
print(f't4 = {t4}')
print(f't5 = {t5}')
print(f't6 = {t6}')

t2 = tensor([[1, 2, 3],
        [4, 5, 6]])
t3 = tensor([[1, 3],
        [4, 6],
        [7, 9]])
t4 = tensor([2, 9, 5])
t5 = tensor([[1, 2, 3],
        [4, 5, 6]])
t6 = tensor([2, 4, 6, 8])


- ブロードキャストを使った演算
- ブロードキャストが起きる条件
<br>2つのテンソルの軸数が1以上
<br>2つのテンソルを最終軸から比較した場合、各軸の次元が同じであるか。どちらかが1であるか、どちらかの軸が存在しない

In [12]:
t1 = torch.tensor([1, 2]).view(2, 1)
t2 = torch.tensor([3, 4, 5])

t3 = t1 + t2

t4 = torch.tensor([1, 2, 3, 4, 5, 6]).view(3, 2)
t5 = torch.tensor([3, 4])

t6 = t4 + t5

print(f't3 = {t3}')
print(f't6 = {t6}')

t3 = tensor([[4, 5, 6],
        [5, 6, 7]])
t6 = tensor([[ 4,  6],
        [ 6,  8],
        [ 8, 10]])


- PyTorchのモジュール

In [16]:
# 多クラスロジスティック回帰モデル
class MultiClassLogisticRegression(nn.Module):
    def __init__(self, dim_input: int, num_class: int):
        super().__init__()
        
        self.linear = nn.Linear(dim_input, num_class)
        
    """
    順伝播関数
    """
    def forward(self, x: torch.Tensor):
        l = self.linear(x)
        y = l.softmax(dim=1)
        
        return y

In [18]:
# 多クラスロジスティック回帰モデルの使用例
model = MultiClassLogisticRegression(32 * 32 * 3, 10)

# 学習モード
model.train()

# 推論モード
model.eval()

x = torch.normal(0, 1, size=(1, 32 * 32 * 3))
y = model(x)

for name, parameter in model.named_parameters():
    print(f'{name}: shape = {parameter.shape}')

linear.weight: shape = torch.Size([10, 3072])
linear.bias: shape = torch.Size([10])


- Sequentialクラス
<br>複数の処理を直列にまとめて適用するためのクラス

In [19]:
class FNNSequential(nn.Module):
    def __init__(self, dim_input: int, num_classes: int):
        super().__init__()
        
        self.layers = nn.Sequential(
            nn.Linear(dim_input, 256),
            nn.ReLU(inplace=True),
            nn.Linear(256, 256),
            nn.ReLU(inplace=True),
            nn.Linear(256, 256),
            nn.ReLU(inplace=True),
            nn.Linear(256, num_classes)
        )
        
    def forward(self, x):
        l = self.layers(x)
        y = l.softmax(dim=1)
        
        return y

- ModuleList
<br>複数の処理をリストにまとめて保存しておく

In [20]:
class FNNModuleList(nn.Module):
    def __init__(self, dim_input: int, num_classes: int):
        super().__init__()
        
        layers = [nn.Linear(dim_input, 256)]
        layers += [nn.Linear(256, 256) for _ in range(2)]
        layers.append(nn.Linear(256, num_classes))
        self.layers = nn.ModuleList(layers) # ModuleListクラスを使う
        
    def forward(self, x):
        for layer in self.layers[:-1]:
            x = F.relu(layer(x))
        l = self.layers[-1](x)
        y = l.softmax(dim=1)
        
        return y

In [21]:
model_sequential = FNNSequential(32 * 32 * 3, 10)
model_modulelist = FNNModuleList(32 * 32 * 3, 10)

model_sequential.eval()
model_modulelist.eval()

x = torch.normal(0, 1, size=(1, 32 * 32 * 3))
y_sequential = model_sequential(x)
y_modulelist = model_modulelist(x)

print(f'y_sequential = {y_sequential}')
print(f'y_modulelist = {y_modulelist}')

y_sequential = tensor([[0.1028, 0.1018, 0.0925, 0.1045, 0.1017, 0.1001, 0.1001, 0.0970, 0.1051,
         0.0944]], grad_fn=<SoftmaxBackward0>)
y_modulelist = tensor([[0.1074, 0.1023, 0.1003, 0.0939, 0.1049, 0.0999, 0.0962, 0.0981, 0.0926,
         0.1043]], grad_fn=<SoftmaxBackward0>)


- 自動微分
<br>PyTorchがグラフ構造と順伝播時の個々の計算処理の勾配計算を記録している

In [22]:
linear = nn.Linear(32 * 32 * 3, 10)

# 入力ベクトル
x = torch.normal(0, 1, size=(1, 32 * 32 * 3))
y = torch.tensor([0])

# 目的関数（交差エントロピー誤差）の計算
y_pred = linear(x)
loss = F.cross_entropy(y_pred, y)

# 誤差逆伝播（自動微分）
loss.backward()

print(f'linear.weight.grad = {linear.weight.grad}')
print(f'linear.bias.grad = {linear.bias.grad}')

linear.weight.grad = tensor([[ 0.9379, -0.4124, -0.3194,  ..., -0.6923,  0.3473,  0.2563],
        [-0.0905,  0.0398,  0.0308,  ...,  0.0668, -0.0335, -0.0247],
        [-0.1278,  0.0562,  0.0435,  ...,  0.0943, -0.0473, -0.0349],
        ...,
        [-0.0537,  0.0236,  0.0183,  ...,  0.0397, -0.0199, -0.0147],
        [-0.0665,  0.0292,  0.0226,  ...,  0.0491, -0.0246, -0.0182],
        [-0.0715,  0.0315,  0.0244,  ...,  0.0528, -0.0265, -0.0196]])
linear.bias.grad = tensor([-0.9625,  0.0929,  0.1312,  0.2564,  0.1196,  0.0751,  0.0906,  0.0552,
         0.0682,  0.0734])
